In [1]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2

import os
print(os.getcwd())
def update_working_directory():
    from pathlib import Path
    p = Path(os.getcwd()).parents[0]
    os.chdir(p)
    print(p)
update_working_directory()

/Users/admin/Projects/vocabulary_learning/notebooks
/Users/admin/Projects/vocabulary_learning


In [2]:
import numpy as np
import pandas as pd

In [3]:
def get_vocab(vocab_path='data/raw/german_english.csv', list_columns=None):
    vocab = pd.read_csv(vocab_path)

    if list_columns is None:
        list_columns = ['id_vocab', 'german', 'english']
    vocab = vocab[list_columns]

    return vocab

In [5]:
vocab = get_vocab('data/raw/german_english__feature.csv')
vocab

,id_vocab,german,english
0,1,rennen,to sprint
1,2,richtig,correct
2,3,September,September
3,4,sprechen,to speak
4,5,stimmen,to vote
5,6,verheiratet,married
6,7,wirklich,really
7,8,die Aprikose,the apricot
8,9,das Jahr,the year
9,10,viel,a lot


# Overall

In [7]:
from src.data.make_vocab_features import create_vocab_features

In [8]:
vocab_test = create_vocab_features(vocab)
vocab_test

,id_vocab,german,english,levenshtein_dist
0,1,rennen,to sprint,6
1,2,richtig,correct,7
2,3,September,September,0
3,4,sprechen,to speak,5
4,5,stimmen,to vote,6
5,6,verheiratet,married,8
6,7,wirklich,really,7
7,8,die Aprikose,the apricot,3
8,9,das Jahr,the year,3
9,10,viel,a lot,5


# Levenshtein distance between words

In [ ]:
def remove_article(
    vocab,
    list_german_article = ['der','die','das'],
    list_english_article = ['the','to']
):

    vocab['german'] = vocab['german'].map(
        lambda x: ' '.join(word for word in x.split(' ') if word not in list_german_article)
    )
    vocab['english'] = vocab['english'].map(
        lambda x: ' '.join(word for word in x.split(' ') if word not in list_english_article)
    )

In [ ]:
def add_levenshtein_distance(vocab):
    
    from Levenshtein import distance
    
    vocab = vocab.copy()
    
    # Lowercase
    vocab['german'] = vocab['german'].str.lower()
    vocab['english'] = vocab['english'].str.lower()
    
    # Remove article
    remove_article(vocab)
    
    # Calculate Levenshtein distance
    levenshtein_dist = vocab.apply(lambda x: distance(x['german'], x['english']), axis=1)
    
    return levenshtein_dist

In [ ]:
vocab['levenshtein_dist'] = add_levenshtein_distance(vocab)
vocab

In [ ]:
# use dill - works the same way as pickle
import dill
with open('data/processed/vocab.pkl', 'wb') as file:
    dill.dump(vocab, file)

# Tests

In [ ]:
def assert_vocab_remove_articles():

    # Creating test dataframe
    vocab_test = pd.DataFrame(columns=['german', 'english'])
    vocab_test = vocab_test.append({'german': 'dienstag', 'english': 'tuesday'}, ignore_index=True)
    vocab_test = vocab_test.append({'german': 'studieren', 'english': 'to study'}, ignore_index=True)
    vocab_test = vocab_test.append({'german': 'die angst', 'english': 'the fear'}, ignore_index=True)
    vocab_test = vocab_test.append({'german': 'andere', 'english': 'other'}, ignore_index=True)
    vocab_test = vocab_test.append({'german': 'der arzt', 'english': 'the doctor'}, ignore_index=True)
    vocab_test = vocab_test.append({'german': 'das café', 'english': 'the café'}, ignore_index=True)

    # Creating test dataframe
    vocab_results = pd.DataFrame(columns=['german', 'english'])
    vocab_results = vocab_results.append({'german': 'dienstag', 'english': 'tuesday'}, ignore_index=True)
    vocab_results = vocab_results.append({'german': 'studieren', 'english': 'study'}, ignore_index=True)
    vocab_results = vocab_results.append({'german': 'angst', 'english': 'fear'}, ignore_index=True)
    vocab_results = vocab_results.append({'german': 'andere', 'english': 'other'}, ignore_index=True)
    vocab_results = vocab_results.append({'german': 'arzt', 'english': 'doctor'}, ignore_index=True)
    vocab_results = vocab_results.append({'german': 'café', 'english': 'café'}, ignore_index=True)

    return vocab_test, vocab_results

In [ ]:
assert_vocab_remove_articles()